<a href="https://colab.research.google.com/github/vasantbala/vb_ai_course/blob/main/NLP/Project/NLP_Project_Rough.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Import the libraries, load dataset, print shape of data, data description

In [1]:
!pip install contractions

import re, string, unicodedata                          # Import Regex, string and unicodedata.
import contractions                                     # Import contractions library.
from bs4 import BeautifulSoup                           # Import BeautifulSoup.

import numpy as np                                      # Import numpy.
import pandas as pd                                     # Import pandas.
import nltk                                             # Import Natural Language Tool-Kit.

nltk.download('stopwords')                              # Download Stopwords.
nltk.download('punkt')
nltk.download('wordnet')

from nltk.corpus import stopwords                       # Import stopwords.
from nltk.tokenize import word_tokenize, sent_tokenize  # Import Tokenizer.
from nltk.stem.wordnet import WordNetLemmatizer         # Import Lemmatizer.

     |████████████████████████████████| 266kB 7.4MB/s 
     |████████████████████████████████| 327kB 15.3MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85391 sha256=122615a154a4b5d2e13e73e558c1be90b75c142c5274068ac760faf604c62b5a
  Stored in directory: /root/.cache/pip/wheels/3a/03/34/77e3ece0bba8b86bfac88a79f923b36d805cad63caeba38842
Successfully built pyahocorasick
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [2]:
# load dataset
data = pd.read_csv('https://github.com/vasantbala/vb_ai_course/blob/main/NLP/Project/Tweets.csv?raw=true')

In [4]:
# print shape of data
data.shape

(14640, 15)

In [6]:
# data description
data.describe()

,tweet_id,airline_sentiment_confidence,negativereason_confidence,retweet_count
count,1.464000e+04,14640.000000,10522.000000,14640.000000
mean,5.692184e+17,0.900169,0.638298,0.082650
std,7.791112e+14,0.162830,0.330440,0.745778
min,5.675883e+17,0.335000,0.000000,0.000000
25%,5.685592e+17,0.692300,0.360600,0.000000
50%,5.694779e+17,1.000000,0.670600,0.000000
75%,5.698905e+17,1.000000,1.000000,0.000000
max,5.703106e+17,1.000000,1.000000,44.000000


In [7]:
data.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


## 2. Understand of data-columns

### 2.a Drop all other columns except “text” and “airline_sentiment”.


In [61]:
trimmed_data = data.copy(deep=True)
for col in data.columns:
  if (col != 'airline_sentiment' and col != 'text'):
    trimmed_data.drop([col], axis=1, inplace=True)

### 2.b Check the shape of data

In [62]:

trimmed_data.shape

(14640, 2)

### 2.c Print first 5 rows of data.

In [63]:

trimmed_data.head()

,airline_sentiment,text
0,neutral,@VirginAmerica What @dhepburn said.
1,positive,@VirginAmerica plus you've added commercials t...
2,neutral,@VirginAmerica I didn't today... Must mean I n...
3,negative,@VirginAmerica it's really aggressive to blast...
4,negative,@VirginAmerica and it's a really big bad thing...


## 3. Text pre-processing: Data preparation

### 3.a Html tag removal.

In [64]:
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

trimmed_data['text'] = trimmed_data['text'].apply(lambda x: strip_html(x))
trimmed_data.head()

,airline_sentiment,text
0,neutral,@VirginAmerica What @dhepburn said.
1,positive,@VirginAmerica plus you've added commercials t...
2,neutral,@VirginAmerica I didn't today... Must mean I n...
3,negative,@VirginAmerica it's really aggressive to blast...
4,negative,@VirginAmerica and it's a really big bad thing...


In [65]:
def replace_contractions(text):
    """Replace contractions in string of text"""
    return contractions.fix(text)

trimmed_data['text'] = trimmed_data['text'].apply(lambda x: replace_contractions(x))
trimmed_data.head()

,airline_sentiment,text
0,neutral,@VirginAmerica What @dhepburn said.
1,positive,@VirginAmerica plus you have added commercials...
2,neutral,@VirginAmerica I did not today... Must mean I ...
3,negative,@VirginAmerica it is really aggressive to blas...
4,negative,@VirginAmerica and it is a really big bad thin...


### 3.b Tokenization

In [66]:
trimmed_data['text'] = trimmed_data.apply(lambda row: nltk.word_tokenize(row['text']), axis=1) # Tokenization of data

In [67]:
for i in trimmed_data['text'][0]:
  print(i)

@
VirginAmerica
What
@
dhepburn
said
.


### 3.c Remove the numbers

In [68]:
trimmed_data['text'][0]

['@', 'VirginAmerica', 'What', '@', 'dhepburn', 'said', '.']

In [69]:
def remove_numbers(textList):
  for tokenIndex in range(0, len(textList)):
    textList[tokenIndex] = re.sub(r'\d+', '', textList[tokenIndex])
  return textList

trimmed_data['text'] = trimmed_data['text'].apply(lambda x: remove_numbers(x))


In [70]:
trimmed_data.head()

,airline_sentiment,text
0,neutral,"[@, VirginAmerica, What, @, dhepburn, said, .]"
1,positive,"[@, VirginAmerica, plus, you, have, added, com..."
2,neutral,"[@, VirginAmerica, I, did, not, today, ..., Mu..."
3,negative,"[@, VirginAmerica, it, is, really, aggressive,..."
4,negative,"[@, VirginAmerica, and, it, is, a, really, big..."


### 3.d Removal of Special Characters and Punctuations

In [71]:
def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words
trimmed_data['text'] = trimmed_data['text'].apply(lambda x: remove_punctuation(x))    

### 3.e Conversion to lowercase

In [72]:
def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words
trimmed_data['text'] = trimmed_data['text'].apply(lambda x: to_lowercase(x))    
trimmed_data.head()

,airline_sentiment,text
0,neutral,"[virginamerica, what, dhepburn, said]"
1,positive,"[virginamerica, plus, you, have, added, commer..."
2,neutral,"[virginamerica, i, did, not, today, must, mean..."
3,negative,"[virginamerica, it, is, really, aggressive, to..."
4,negative,"[virginamerica, and, it, is, a, really, big, b..."


### 3.f Lemmatize or stemming

In [73]:
lemmatizer = WordNetLemmatizer()
def lemmatize_list(words):
    new_words = []
    for word in words:
      new_words.append(lemmatizer.lemmatize(word, pos='v'))
    return new_words
trimmed_data['text'] = trimmed_data['text'].apply(lambda x: lemmatize_list(x))    
trimmed_data.head()    

,airline_sentiment,text
0,neutral,"[virginamerica, what, dhepburn, say]"
1,positive,"[virginamerica, plus, you, have, add, commerci..."
2,neutral,"[virginamerica, i, do, not, today, must, mean,..."
3,negative,"[virginamerica, it, be, really, aggressive, to..."
4,negative,"[virginamerica, and, it, be, a, really, big, b..."


### 3.g Join the words in the list to convert back to text string in the dataframe. (So that each row contains the data in text format.)


In [74]:
def normalize(words):   
    return ' '.join(words)
trimmed_data['text'] = trimmed_data['text'].apply(lambda x: normalize(x))    

,airline_sentiment,text
0,neutral,virginamerica what dhepburn say
1,positive,virginamerica plus you have add commercials to...
2,neutral,virginamerica i do not today must mean i need ...
3,negative,virginamerica it be really aggressive to blast...
4,negative,virginamerica and it be a really big bad thing...


### 3.h Print first 5 rows of data after pre-processing


In [75]:
trimmed_data.head()  

,airline_sentiment,text
0,neutral,virginamerica what dhepburn say
1,positive,virginamerica plus you have add commercials to...
2,neutral,virginamerica i do not today must mean i need ...
3,negative,virginamerica it be really aggressive to blast...
4,negative,virginamerica and it be a really big bad thing...


## 4. Vectorization

### 4.a Use CountVectorizer

### 4.b Use TfidfVectorizer

## 5. Fit and evaluate model using both type of vectorization

## 6. Summarize your understanding of the application of Various Pre-processing and Vectorization and performance of your model on this dataset